<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#NC-Data-Prototyping" data-toc-modified-id="NC-Data-Prototyping-1">NC Data Prototyping</a></span><ul class="toc-item"><li><span><a href="#Loading-the-data" data-toc-modified-id="Loading-the-data-1.1">Loading the data</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Current-voter-list" data-toc-modified-id="Current-voter-list-1.1.0.1">Current voter list</a></span></li><li><span><a href="#Voter-history" data-toc-modified-id="Voter-history-1.1.0.2">Voter history</a></span></li></ul></li><li><span><a href="#Joining-dataframes" data-toc-modified-id="Joining-dataframes-1.1.1">Joining dataframes</a></span></li><li><span><a href="#Still-need-to-figure-out:" data-toc-modified-id="Still-need-to-figure-out:-1.1.2">Still need to figure out:</a></span></li></ul></li><li><span><a href="#New-approach-for-distance-calculation" data-toc-modified-id="New-approach-for-distance-calculation-1.2">New approach for distance calculation</a></span></li></ul></li></ul></div>

# NC Data Prototyping

## Loading the data

In [1006]:
# imports
import dask.dataframe as dd
import numpy as np
import pandas as pd
import re

There are two data files: 
 - voter (info about the voter)
 - voterhistory (whether or not they voted)

#### Current voter list

The data is a 3.2GB text file that is tsv. It takes forever to load the full thing, so I'm only loading the first 100,000 rows for now. Note - I also shuffled the order of the rows to make sure I'm not missing systematic errors that are only happening in one place.   

In [1007]:
voter = pd.read_csv('ncvoter.txt', sep='\t', encoding = "ISO-8859-1", nrows=100000).sample(frac=1)
voter.head(3)

,county_id,county_desc,voter_reg_num,status_cd,voter_status_desc,reason_cd,voter_status_reason_desc,absent_ind,name_prefx_cd,last_name,...,munic_dist_desc,dist_1_abbrv,dist_1_desc,dist_2_abbrv,dist_2_desc,confidential_ind,birth_year,ncid,vtd_abbrv,vtd_desc
85117,1,ALAMANCE,9106230,R,REMOVED,RD,DECEASED,,,SAYERS,...,NaN,NaN,NaN,,,N,1945,AX48629,NaN,NaN
10661,1,ALAMANCE,9098964,A,ACTIVE,AV,VERIFIED,,,BRIDGES,...,BURLINGTON,17.0,17TH PROSECUTORIAL,,,N,1955,AA147731,03S,03S
92880,1,ALAMANCE,9102014,I,INACTIVE,IU,CONFIRMATION RETURNED UNDELIVERABLE,,,STRADER,...,BURLINGTON,17.0,17TH PROSECUTORIAL,,,N,1977,BL319567,03S,03S


#### Voter history

In [1008]:
vhis_unfiltered = pd.read_csv('ncvhis.txt', sep='\t', encoding = "ISO-8859-1", nrows=200000).sample(frac=1)
vhis_unfiltered.head(3)

,county_id,county_desc,voter_reg_num,election_lbl,election_desc,voting_method,voted_party_cd,voted_party_desc,pct_label,pct_description,ncid,voted_county_id,voted_county_desc,vtd_label,vtd_description
138198,89,TYRRELL,5078,11/08/2016,11/08/2016 GENERAL,ABSENTEE ONESTOP,UNA,UNAFFILIATED,2,COLUMBIA,AA151516,89,TYRRELL,2,2
60431,1,ALAMANCE,9076366,11/06/2012,11/06/2012 GENERAL,ABSENTEE ONESTOP,REP,REPUBLICAN,09N,NORTH THOMPSON,AA125584,1,ALAMANCE,09N,09N
96978,1,ALAMANCE,9088485,07/17/2012,07/17/2012 SECOND PRIMARY,IN-PERSON,REP,REPUBLICAN,10N,NORTH MELVILLE,AA137703,1,ALAMANCE,10N,10N


In [1009]:
# Filtering to only include 2012 and 2016 general elections
vhis_2012 = vhis_unfiltered[vhis_unfiltered['election_desc'] == '11/06/2012 GENERAL']
vhis_2016 = vhis_unfiltered[vhis_unfiltered['election_desc'] == '11/08/2016 GENERAL']

##### 2012 voter history data

In [1010]:
# Left joining the tables (have to join it first with 2012 voter history first, and then with 2016)
joined_2012 = pd.merge(voter, vhis_2012, on='voter_reg_num', how='left')

In [1011]:
# Filtering out unnecessary columns
joined_2012 = joined_2012[['voter_reg_num', 'voter_status_desc', 'res_street_address', 
                          'res_city_desc', 'state_cd', 'zip_code', 'race_code', 'voting_method', 
                          'precinct_abbrv', 'precinct_desc', 'vtd_label']]

In [1012]:
# Renaming columns  
joined_2012 = joined_2012.rename(columns = {'voting_method': '2012_voting_method',
                                 'precinct_abbrv': '2016_pct_assignment_abbrv',
                                 'precinct_desc': '2016_pct_assignment_desc'})

##### 2016 voter history data

In [1013]:
# Joining the 2012/voter data with the 2016 data 
joined_2016 = pd.merge(joined_2012, vhis_2016, on='voter_reg_num', how='left')

In [1014]:
# Renaming columns from 2016
joined_2016 = joined_2016.rename(columns = {'pct_description': '2016_pct_name',
                                 'vtd_label': '2016_pct_code',
                                    'voting_method': '2016_voting_method'})

In [1015]:
# Filtering to necessary columns
df = joined_2016[['voter_reg_num', 'voter_status_desc', 'res_street_address',
       'res_city_desc', 'state_cd', 'zip_code', 'race_code', '2016_pct_assignment_abbrv', 
       '2016_pct_assignment_desc', '2012_voting_method', '2016_voting_method']]

##### Adding 2012 voter history data

In [1016]:
# Load everything into dask, filter there, then export to pandas?

In [1017]:
voter_2012 = pd.read_csv('VR_Snapshot_20121106.txt', sep='\t', 
                         encoding = "UTF-16", nrows=200000, low_memory=False).sample(frac=1)
voter_2012.head(3)

,snapshot_dt,county_id,county_desc,voter_reg_num,ncid,status_cd,voter_status_desc,reason_cd,voter_status_reason_desc,absent_ind,...,dist_1_abbrv,dist_1_desc,dist_2_abbrv,dist_2_desc,confidential_ind,cancellation_dt,vtd_abbrv,vtd_desc,load_dt,age_group
193967,2012-11-06,10,BRUNSWICK,11031,AK11282,R,REMOVED,RP,REMOVED UNDER OLD PURGE LAW,,...,,NaN,,NaN,N,NaN,,NaN,2012-12-11,Age 41 - 65
106983,2012-11-06,1,ALAMANCE,3626100,AA24922,R,REMOVED,RD,DECEASED,,...,,NaN,,NaN,N,NaN,,NaN,2012-12-11,Age 41 - 65
184179,2012-11-06,1,ALAMANCE,5009400,AA33663,A,ACTIVE,AV,VERIFIED,,...,15A,15A PROSECUTORIAL,,NaN,N,NaN,035,035,2012-12-11,Age Over 66


In [1018]:
# Filtering to necessary columns

voter_2012 = voter_2012[['voter_reg_num', 'voter_status_desc', 'house_num','street_dir', 
            'street_name', 'street_type_cd', 'res_city_desc', 'state_cd', 'zip_code', 
            'precinct_abbrv', 'precinct_desc']]

In [1019]:
# Getting rid of people who weren't registered in 2012  
voter_2012 = voter_2012[voter_2012['precinct_desc'].notnull()]

In [1020]:
def address_fixer (df):
    """Takes a dataframe with address spread across multiple columns and creates a new
    column with the full, combined address"""
    
    
    house_number = str(df['house_num'])
    direction = df['street_dir']
    street_name = df['street_name']
    street_type = df['street_type_cd']  

    unstripped_address = '{} {} {} {}'.format(house_number, direction, street_name, street_type)
    address = re.sub('\s+', ' ', unstripped_address).strip()
    return address    

In [1021]:
# Removing some unnecessary columns
voter_2012 = voter_2012[['voter_reg_num', 'house_num', 'res_city_desc',
                       'state_cd', 'zip_code', 'precinct_abbrv', 'precinct_desc']]

In [1022]:
voter_2012 = voter_2012.rename(columns = {'house_num': '2012_house_num', 
                               'precinct_abbrv': '2012_pct_assignment_abbrv', 
                               'precinct_desc': '2012_pct_assignment_desc'}, )

### Joining dataframes

In [1023]:
def house_num_finder (df):
    
    """Extracts only the house number from an address (since this is easier to match 
    on to check is someone moved). First checks to make sure it's for an active record""" 
    
    if df['voter_status_desc'] == 'ACTIVE': 
        full_address = df['res_street_address'] 
        return int(re.findall('\d+', full_address)[0])

In [1024]:
# Joining the tables
both_years_df = pd.merge(df, voter_2012, on='voter_reg_num')
both_years_df['2016_house_num'] = both_years_df.apply(house_num_finder, axis=1)

In [1025]:
# Removing people who moved homes between 2012 and 2016
both_years_df = both_years_df[(both_years_df['2012_house_num'] == both_years_df['2016_house_num']) 
                   & (both_years_df['voter_status_desc'] == 'ACTIVE')]

In [1026]:
def changed_polling_location (df):
    
    """Indicates whether someone was moved to a different
    polling place between 2012 general election and 2016 general election."""
    
    if str(df['2016_pct_assignment_abbrv']).strip() != str(df['2012_pct_assignment_abbrv']).strip():
        return 1
    else:
        return 0

In [1027]:
# Running the function to see whose polling location changed
both_years_df['changed_precincts'] = both_years_df.apply(changed_polling_location, axis=1)

In [1028]:
mean = both_years_df['changed_precincts'].mean()
print ('Percent of voters whose polling location changed between 2012 and 2016: ' + '{:.3f}'.format(mean*100) + '%')

Percent of voters whose polling location changed between 2012 and 2016: 0.109%


### Still need to figure out:
 - This works above, but I need to figure out how to run it on the entire dataset. Hoping to use pandas rather than dask but we'll see  

## New approach for distance calculation

_Idea_:
 - 1) Get a list of every voter ID who was assigned to a different polling place (excluding people who moved) 
 - 2) Find the _K_ nearest neighbors to that person whose polling place was moved, via Lat/Long points, and for each of them, determine whether their polling place was also moved. There should be a max threshold - e.g., if the closest neighbor is 8 miles away, it doesn't hold
 - 3) From there, maybe we compare the person whose location was changed to the closest person whose location was not changed, or maybe we compare the person to the X closest people whose weren't changed      